In [ ]:
from copy import deepcopy

import numpy as np
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

# Setup

Set up a log likelihood function for each wimp mass. We could also have set up one likelihood function with many wimps, it doesn't make much difference.

(this approach is less memory-efficient as we have more pdfs in RAM, but a bit more efficient in CPU: having many WIMPs per model means you have to score each event for many sources)

In [ ]:
from wimpy.xenon.base_model import config as base_config
from wimpy.xenon.base_model import nr_ignore_settings
from wimpy.likelihood import LogLikelihood
from scipy import stats

lfs = {}
wimp_masses = [10, 20, 50, 100, 1000]
# Sample low-mass wimps more, since most of their events are out of range
# Maybe we can cut off their spectra at 0.5 keV or something.
# 6 GeV would require something like a 250x multiplier... too impatient
sampling_multipliers = {10: 10, 20:2}   # Maybe 5x is overkill, but ok

for m in wimp_masses:    
    # Copy the base config and remove the wimps present there
    config = deepcopy(base_config)
    config['pdf_sampling_multiplier'] = 2
    config['sources'] = [s for s in config['sources'] if not s['name'].startswith('wimp')]
    
    # Add this WIMP
    config['sources'].append({
        'energy_distribution': 'wimp_%dgev_1e-45.pkl' % m,   # TODO: generate and save on the fly
        'color': 'red',
        'recoil_type': 'nr',
        'name': 'wimp_%dgev' % m,
        'n_events_for_pdf': 5e6 * sampling_multipliers.get(m, 1),
        'ignore_settings': nr_ignore_settings,
        'label': '%d GeV WIMP' % m})
    
    # Create the log likelihood function
    lf = LogLikelihood(config)
    lf.add_rate_parameter('wimp_%dgev' % m)
    lf.add_shape_parameter('leff', 
                           {t: 'leff_mcpaper_%s.csv' % t for t in [-2, -1, 0, 1, 2]}, 
                           log_prior=stats.norm.logpdf)
    lf.prepare()
    
    # Store it in the dictionary
    lfs[m] = lf

Here is the function that will set a WIMP limit. We'll work with the "wimp strength" variable, defined as 

    log10(cross section / 1e-45 cm^2)
    
or equivalently

    log10(event rate / rate for a wimp with 1e-45 cm^2 cross section)
    
Here `event_rate` meant the rate of *all* wimp interactions in the detector, not just the ones that produce events in our analysis range! The fraction of events in range depends on the wimp mass and Leff.

It uses the `bestfit_scipy` function from wimpy.analysis, which simply makes our likelihood function talk to the scipy minimize. 

In [ ]:
from scipy.optimize import brentq
from scipy import stats

from wimpy.analysis import bestfit_scipy as bestfit

def wimplimit(lf, profile=True, confidence_level=0.9, target=None, ndf=1, **kwargs):
    """Computes a wimp limit using the LogLikelihood function lf
    profile: whether or not to profile over the other parameters of lf.
    kwargs are passed to bestfit_scipy (and eventually to lf).
    """
    if target is None:
        target = lf.source_list[-1]
    base_rate = lf.base_model.get_source(target).events_per_day
        
    # Find the likelihood of the global best fit (denominator of likelihood ratio)
    target = '%s_rate' % target
    result, max_loglikelihood = bestfit(lf, **kwargs)
    # A strength of less -6 means we never see a wimp; the difference with -inf (or whatever it is) is insignificant
    # Forgot exactly what problems you get into if you don't do this... but there were some
    best_strength = max(-6, np.log10(result[target] / base_rate))
    
    # Find the best fit conditional to a wimp strength (numerator of likelihood ratio)
    def f(wimp_strength, critical_chi2):
        lf_kwargs = {target: base_rate * 10**wimp_strength}
        lf_kwargs.update(kwargs)
        if profile:
            fitresult, ll= bestfit(lf, **lf_kwargs)
        else:
            ll = lf(**lf_kwargs)
        result = np.sqrt(2*(max_loglikelihood - ll)) - critical_chi2
        return result
        
    # Perform a line search 
    return brentq(f, best_strength, max(best_strength + 2, 2),
                  args=(stats.chi2(ndf).ppf(0.9),))

# Load background data

I load a standard dataset of 1000 background samples, so I can study and refer to particular problematic ones.

In [ ]:
bg_filename = 'bg_dsets.pkl'

# n_trials = 1000
# bg_dsets = [lfs[50].base_model.simulate(rate_multipliers=dict(wimp_50gev=0)) 
#             for _ in tqdm(range(n_trials))]
# import pickle
# with open(bg_filename, mode='wb') as outfile:
#     pickle.dump(bg_dsets, outfile)

with open(bg_filename, mode='rb') as infile:
    bg_dsets = pickle.load(infile)

Here is one background-only dataset. As it happens this contains a single neutron and no CNNS events.

In [ ]:
d = bg_dsets[0]
lf = lfs[50]
lf.base_model.show(d)
plt.legend(loc='lower right', scatterpoints=1)

Let's see what the log likelihood function (for a 50 GeV WIMP) thinks of this:

In [ ]:
lf.set_data(d)
plot_likelihood_ratio(lf, ('wimp_50gev_rate', np.logspace(-4, 1, 100)), ('leff', np.linspace(-2, 2, 100)))
plt.xscale('log')

In [ ]:
def sigma_to_p(sigma):
    return stats.norm.cdf(sigma) - stats.norm.cdf(-sigma)

In [ ]:
from wimpy.analysis import plot_likelihood_ratio

base_rate = lf.base_model.get_source('wimp_50gev').events_per_day

for leff in [2, 0, -2]:
    plot_likelihood_ratio(lf, ('wimp_50gev_rate', np.logspace(-4, 0, 50)), leff=leff, 
                          plot_kwargs=dict(label="Leff t=%s" % leff), vmax=5)
    # strength = wimplimit(lf, leff=leff, profile=False)

plt.axhline(stats.chi2(1).ppf(0.9)**2/2, linestyle='--', label='90% asymptotic', c='gray')
plt.xscale('log')
plt.legend(loc='upper left')

In [ ]:
# %%timeit
# lfs[50](wimp_50_gev_rate=1, leff=0.1)
# 0.5 ms / likelihood function call
# Could probably do a small brute force search to seed the minimizer

In [ ]:
bestfit(lf, guess=dict(wimp_50gev_rate=0))

In [ ]:
bestfit(lf, guess=dict(wimp_50gev_rate=100))

In [ ]:
limits_lr = {m: [] for m in wimp_masses}
limits_plr = {m: [] for m in wimp_masses}
limits_lr_lowleff = {m: [] for m in wimp_masses}

for m in wimp_masses:
    lf = lfs[m]
    
    for i, d in enumerate(tqdm(bg_dsets)):
        lf.set_data(d)
        guess = {'wimp_%dgev_rate' % wimp_mass: 1e-2}
        limits_lr[m].append(wimplimit(lf, profile=False, leff=0, guess=guess))
        limits_lr_lowleff[m].append(wimplimit(lf, profile=False, leff=-2, guess=guess))
        try:
            limits_plr[m].append(wimplimit(lf, profile=True, guess=guess))
        except Exception as e:
            print("Failure in limit setting for dataset %d, %d GeV: %s, %s" % (i, m, str(type(e)), str(e)))
        
    limits_lr[m] = np.array(limits_lr[m])
    limits_plr[m] = np.array(limits_plr[m])

In [ ]:
# Load the Bologna model data points
import pandas as pd
ms, xs = pd.read_csv('./bologna_nocls_limits/median_nocls.csv', skiprows=5, names=['m', 'xs'])[1:].values.astype(np.float).T
xs += 45
plt.plot(ms, xs, color='k', linestyle=':', label='Bologna model (no CLS)', linewidth=2)

# Plot the median of the limits we just derived, and the bands for the profiled limits
for limit_dict, label, color in ((limits_lr, 'Leff = 0', 'g'),
                                 (limits_lr_lowleff, 'Leff = -2', 'r'),
                                 (limits_plr, 'Leff profiled', 'b')):
    limits_sigma = {sigma: np.array([np.percentile(limit_dict[m], 100*stats.norm.cdf(sigma)) 
                                     for m in wimp_masses])
                    for sigma in [-2, -1, 0, 1, 2]}
    plt.plot(wimp_masses, limits_sigma[0], 
             marker='o', color=color, label=label)
    if color == 'b':
        plt.fill_between(wimp_masses, limits_sigma[-1], limits_sigma[1], alpha=0.1, color=color)
        plt.fill_between(wimp_masses, limits_sigma[-2], limits_sigma[2], alpha=0.03, color=color)

plt.xscale('log')
plt.legend(loc='upper right', numpoints=1, frameon=False)
plt.xlim(7, 1100)
plt.ylim(-2.5, 0.5)
plt.xlabel("Wimp mass (GeV)")
plt.ylabel("Log10 (cross section (zb))")
plt.show()

Leff moves the NR band horizontally; the lower LEff, the further left it goes. This has two effects: it pushes events below threshold (the rate effect) and pushes them nearer to the ER band (the shape effect). Both weaken your sensitivity; the first obivously so, the second because it makes ER leakage events more likely to be interpreted as WIMP events.

In [ ]:
lf= lfs[1000]
for i, ((leff,), m) in enumerate(sorted(lf.anchor_models.items())):
    m.sources[-1].pdf_histogram.percentile(50, axis=1).plot(label='Nr median, Leff %d' % leff, color=plt.cm.Reds(i/4))
   
m.sources[0].pdf_histogram.plot(cblabel='ER PDF', log_scale=True, vmin=1e-9, cmap=plt.cm.viridis)
#m.sources[0].pdf_histogram.percentile().plot(cblabel='ER PDF', log_scale=True, vmin=1e-8, cmap=plt.cm.Blues)
leg = plt.legend(loc='upper left')
leg.get_frame().set_facecolor('lightgray')

# Regularity of likelihood function

In [ ]:
plot_likelihood_ratio(lf, ('leff', np.linspace(-2, 2, 100)), wimp_1000gev_rate=1e-3, vmax=4)

In [ ]:
# Check if pdf differences are well sampled
lf= lfs[1000]
s_i = -1
a = lf.anchor_models[(0,)].sources[s_i]
b = lf.anchor_models[(-1.0,)].sources[s_i]
fr_increase = (b.pdf_histogram - a.pdf_histogram )/a.pdf_histogram
#q = max(abs(fr_increase.histogram.max()), abs(fr_increase.histogram.min()))
#q *= 0.5
q = 1
fr_increase.plot(vmin=-q, vmax=q, cmap=plt.cm.seismic, cblabel="Fractional increase in PDF")